In [1]:
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

In [2]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_core.prompts import PromptTemplate
llm = ChatOpenAI(model = 'gpt-4.1-nano')
prompt = PromptTemplate.from_template(
    '아래 이메일 내용중 중요한 것만 추출해.\n\n---\n\n{email_conversation}'
)

# print(prompt.format(email_conversation=email_conversation))
chain = prompt | llm 

print('---출력파서 없는 요약---')
print(chain.invoke({'email_conversation': email_conversation}).content)

---출력파서 없는 요약---
1. 바이크코퍼레이션의 김철수 상무가 "ZENESIS" 자전거에 관심을 가지고 있으며, 상세 브로슈어 요청  
2. 요청 내용: 기술 사양, 배터리 성능, 디자인 정보 제공 필요  
3. 미팅 일정 제안: 1월 15일(화요일) 오전 10시, 귀사 사무실에서 만남 추진


In [ ]:
from pydantic import BaseModel, Fiel

In [11]:
from langchain.output_parsers import PydanticOutputParser

In [13]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser # 👈 이 줄을 추가하세요

# Pydantic 모델 정의
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='보낸사람의 메일주소')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')

# 파서 객체 생성 (이제 오류 없이 실행됩니다)
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# 파서가 생성한 포맷팅 지침 출력
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "보낸사람의 메일주소", "title": "Email", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "summary", "date"]}
```


In [14]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 답변을 한국어로 만들어줘

질문: {question}

이메일 내용: {email_conversation}

형식: {format}
"""
)

# 프롬프트 변수들중 일부만 채우기
prompt = prompt.partial(format=parser.get_format_instructions())

In [15]:
chain = prompt | llm | parser

res = chain.invoke(
    {
        'question': '이메일 내용중 중요한 내용을 추출해 줘!',
        'email_conversation': email_conversation
    }
)

res

EmailSummary(person='김철수 상무', email='chulsoo.kim@bikecorporation.me', summary="김철수 상무는 귀사의 자전거 'ZENESIS'에 대한 브로슈어 요청과 함께, 기술 사양, 배터리 성능, 디자인 관련 정보를 요청하며, 1월 15일 화요일 오전 10시에 미팅을 제안하여 협력 가능성을 논의하려고 합니다.", date='2023년 1월 15일 오전 10시')

In [16]:
from pprint import pprint
pprint(res.model_dump())


{'date': '2023년 1월 15일 오전 10시',
 'email': 'chulsoo.kim@bikecorporation.me',
 'person': '김철수 상무',
 'summary': "김철수 상무는 귀사의 자전거 'ZENESIS'에 대한 브로슈어 요청과 함께, 기술 사양, 배터리 성능, 디자인 관련 "
            '정보를 요청하며, 1월 15일 화요일 오전 10시에 미팅을 제안하여 협력 가능성을 논의하려고 합니다.'}


In [17]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# CSV 파서의 안내사항 확인
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='List 5 {subject}.\n{format_instructions}',
    input_variables=['subject'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)
print('----')
print(prompt.format(subject='중국집대표메뉴'))

chain1 = prompt | llm
csv_res = chain1.invoke({'subject': '맥도날드 대표메뉴'}).content


chain2 = prompt | llm | output_parser
list_res = chain2.invoke({'subject': '국민주식'})

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
----
List 5 중국집대표메뉴.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [18]:
print(csv_res)
print(list_res)

Big Mac, Quarter Pounder, Chicken McNuggets, Filet-O-Fish, McChicken
['삼성전자', 'SK하이닉스', '현대자동차', 'LG화학', 'POSCO']


In [19]:
from langchain.output_parsers import PandasDataFrameOutputParser
import seaborn as sns
titanic_df = sns.load_dataset('titanic')

In [21]:
parser = PandasDataFrameOutputParser(dataframe=titanic_df)
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [22]:

prompt = PromptTemplate(
    template='Answer the user query.\n{format_instructions}\n{query}',
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()},
)

chain = prompt | llm | parser

res = chain.invoke({'query': 'age 컬럼을 조회해 줘'})
print(res['age'])  # Sereis

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64
